`Data Loading and Exploration`

In [85]:
import torch
import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [ ]:
# configuration variables
DATA_PATH = "data/train.csv"
MODEL_NAME = 'bert-base-uncased'
NUM_LABELS = 2 
MAX_LENGTH = 512
SAMPLE_SIZE = 5000 # lesser = faster training
TEST_SIZE = 0.2
BATCH_SIZE = 8
EPOCHS = 1
OUTPUT_DIR = "models/toxic_classifier"

In [87]:
toxic_df = pd.read_csv(DATA_PATH)

In [88]:
print("Dataset shape:", toxic_df.shape)
toxic_df.head()

Dataset shape: (159571, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [89]:
toxic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [90]:
# class distribution
print(toxic_df['toxic'].value_counts())

toxic
0    144277
1     15294
Name: count, dtype: int64


In [91]:
# sample equally from both classaes
SAMPLES_PER_CLASS = SAMPLE_SIZE // 2

toxic_samples = toxic_df[toxic_df['toxic'] == 1].sample(n=SAMPLES_PER_CLASS, random_state=42)
non_toxic_samples = toxic_df[toxic_df['toxic'] == 0].sample(n=SAMPLES_PER_CLASS, random_state=42)

In [92]:
# combine and shuffle
toxic_df = pd.concat([toxic_samples, non_toxic_samples]).sample(frac=1, random_state=42).reset_index(drop=True)

In [93]:
print(f"\nBalanced sample of {SAMPLE_SIZE} total samples:")
print(toxic_df['toxic'].value_counts())


Balanced sample of 5000 total samples:
toxic
1    2500
0    2500
Name: count, dtype: int64


`Model and Tokenizer Setup`

In [94]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [95]:
# test tokenizer with sample data
sample_texts = ["I am eating", "I am playing"]
sample_tokens = tokenizer(sample_texts, padding=True, truncation=True, max_length=MAX_LENGTH)

print("Sample tokenization:")
print(sample_tokens)

Sample tokenization:
{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}


`Data Preparation`

In [96]:
# prepare features and labels
X = list(toxic_df["comment_text"])
y = list(toxic_df["toxic"])

In [97]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=y, random_state=42
)
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")

Training samples: 4000
Validation samples: 1000


In [98]:
# tokenize the text data
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=MAX_LENGTH)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=MAX_LENGTH)

`Create PyTorch Dataset`

In [99]:
class ToxicDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.encodings["input_ids"])


In [100]:
# create dataset objects
train_dataset = ToxicDataset(X_train_tokenized, y_train)
val_dataset = ToxicDataset(X_val_tokenized, y_val)

print("Sample training item:")
print(train_dataset[5])

Sample training item:
{'input_ids': tensor([  101, 26219, 12155, 10270,  2546,  1006,  1996, 10597, 10047, 26657,
         2094, 10916,  1007, 24639,  7699,  9326,  2153,  1012,  1012,  1012,
         2070,  2111,  2467,  2074,  4025,  2000,  3499,  2037, 14395,  4297,
        11631, 20935,  2000,  2131,  1996,  2488,  1997,  2068,  1012,  2004,
         1037,  2317,  2966,  3460,  1010,  1045,  2572,  6135, 17733,  2011,
         1996,  2236,  2966,  2473,  1006, 13938,  2278,  1007,  1998,  6118,
         2490,  1996,  1005,  2543,  1998,  7987, 14428,  9221,  1005,  3921,
         1997,  1996, 11113, 20872,  2232,  1996, 13938,  2278,  3049,  1012,
         1996, 13938,  2278,  2811,  2436,  3495,  9530,  3490,  6961,  2007,
         1996, 24173,  2015,  2000,  8479,  2037,  2219,  7491, 19377,  2035,
         2058,  1996,  2173,  1011,  2061,  1996, 11113, 20872,  2232,  1996,
        13938,  2278,  4364,  2031,  2296,  2157,  2000,  5454,  1996,  2200,
         2168,  2806,  2065,

`Define Evaluation Metrics`

In [101]:
def compute_metrics(eval_pred):
    """Calculate accuracy, precision, recall, and F1 score"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    recall = recall_score(y_true=labels, y_pred=predictions)
    precision = precision_score(y_true=labels, y_pred=predictions)
    f1 = f1_score(y_true=labels, y_pred=predictions)
    
    return {
        "accuracy": accuracy, 
        "precision": precision, 
        "recall": recall, 
        "f1": f1
    }

`Training Setup and Execution`

In [102]:
# configure training parameters
training_args = TrainingArguments(
    output_dir="output",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [103]:
trainer.train()

/opt/anaconda3/envs/dev/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.306500


TrainOutput(global_step=500, training_loss=0.30645831298828125, metrics={'train_runtime': 625.1578, 'train_samples_per_second': 6.398, 'train_steps_per_second': 0.8, 'total_flos': 1052444221440000.0, 'train_loss': 0.30645831298828125, 'epoch': 1.0})

In [104]:
trainer.evaluate()

/opt/anaconda3/envs/dev/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.2159002721309662,
 'eval_accuracy': 0.928,
 'eval_precision': 0.91796875,
 'eval_recall': 0.94,
 'eval_f1': 0.9288537549407114,
 'eval_runtime': 35.7127,
 'eval_samples_per_second': 28.001,
 'eval_steps_per_second': 3.5,
 'epoch': 1.0}

`Test Model and Save`

In [105]:
test_texts = [
    "That was a good point",  # Non-toxic
    "go to hell"              # Toxic
]

In [106]:
model = model.cpu()

for text in test_texts:
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    
    with torch.no_grad():  
        outputs = model(**inputs)
    
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predictions = predictions.detach().numpy()  # No need for .cpu() since already on CPU
    
    print(f"Text: '{text}'")
    print(f"Non-toxic probability: {predictions[0][0]:.3f}")
    print(f"Toxic probability: {predictions[0][1]:.3f}")
    print("-" * 40)

Text: 'That was a good point'
Non-toxic probability: 0.997
Toxic probability: 0.003
----------------------------------------
Text: 'go to hell'
Non-toxic probability: 0.004
Toxic probability: 0.996
----------------------------------------


In [107]:
# save the trained model
trainer.save_model(OUTPUT_DIR)

`Push Model to Huggingface Hub`

In [108]:
from huggingface_hub import notebook_login

notebook_login()

In [109]:
model.push_to_hub('thebugged/Bert')

README.md: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/thebugged/Bert/commit/339ecaa047b49fd52ea246ee496dec0e27f3af24', commit_message='Upload BertForSequenceClassification', commit_description='', oid='339ecaa047b49fd52ea246ee496dec0e27f3af24', pr_url=None, repo_url=RepoUrl('https://huggingface.co/thebugged/Bert', endpoint='https://huggingface.co', repo_type='model', repo_id='thebugged/Bert'), pr_revision=None, pr_num=None)

In [ ]:
# model = BertForSequenceClassification.from_pretrained("thebugged/Bert")
# model

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e